In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,log_loss
from mlxtend.plotting import plot_decision_regions
import sys, math, os, json, re, random
import scipy.stats as stats
import seaborn as sns
import datetime as dt

from reticulum import AdaptiveBayesianReticulum
from functions import *

# jupyter magic to display plots directly in the notebook
%matplotlib inline

# use vector graphics format for nicer plots
%config Inline.Backend.figure_format = 'svg'

%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [139]:
training = pd.read_excel('data/RfqData.xlsx',sheet_name = 'Training RFQs')
oos = pd.read_excel('data/RfqData.xlsx',sheet_name = 'OOS RFQs')
competition = pd.read_excel('data/RfqData.xlsx',sheet_name = 'Competition RFQs')

In [131]:
competition

,Time,Bond,Side,Notional,Counterparty,MidPrice,QuotedPrice,Competitors
0,31000,Bond_0,Bid,1000,Ctpy_0,82.55,NaN,3
1,31001,Bond_2,Bid,1000,Ctpy_3,110.32,NaN,4
2,31004,Bond_1,Offer,300000,Ctpy_0,126.83,NaN,2
3,31008,Bond_0,Bid,1000,Ctpy_1,82.28,NaN,1
4,31009,Bond_2,Offer,1000,Ctpy_1,109.99,NaN,2


In [134]:
target_mapping = {'MISSED': 0, 'DONE': 1}
side_mapping = {'Offer': -1, 'Bid': 1}

In [135]:
def add_features(df):
    df['delta_from_mid'] = df['QuotedPrice']/df['MidPrice']
    df['delta_from_nextprice'] = df['NextMidPrice']/df['QuotedPrice']
    #df['delta_from_mid'] = abs(df['QuotedPrice']-df['MidPrice'])
    df['Traded'] = df['Traded'].map(target_mapping)
    df['Side'] = df['Side'].map(side_mapping)
    cols = ['Counterparty','Bond']    
    one_hot_encode = pd.get_dummies(df[cols])
    df = df.join(one_hot_encode)
    #df['Notional_cuts'] = pd.cut(df['Notional'], [0, 100000, 1000000, 10000000, 50000000], labels=[0, 1, 2, 3], include_lowest=True, right=True)
    #df['PnL']=((df['NextMidPrice']-df['QuotedPrice']>0).astype(float)*2-1)*df['Side']
    cols_to_drop = ['Time','Bond','NextMidPrice','Counterparty','Bond']
    return df.drop(columns=cols_to_drop)

In [140]:
train = add_features(training)
test = add_features(oos)
train.head()

,Side,Notional,MidPrice,QuotedPrice,Competitors,Traded,delta_from_mid,delta_from_nextprice,Counterparty_Ctpy_0,Counterparty_Ctpy_1,Counterparty_Ctpy_2,Counterparty_Ctpy_3,Bond_Bond_0,Bond_Bond_1,Bond_Bond_2
0,-1,10000000,124.01,124.25,1,0,1.001935,0.999920,1,0,0,0,0,0,1
1,1,1000,98.07,98.06,1,1,0.999898,1.000204,0,1,0,0,1,0,0
2,-1,1000,170.30,170.40,1,0,1.000587,1.001408,0,1,0,0,0,1,0
3,1,20000,98.00,97.98,4,1,0.999796,0.999592,1,0,0,0,1,0,0
4,-1,1000,171.12,171.16,2,1,1.000234,1.001753,0,0,0,1,0,1,0


#### First Model - Predict Traded

In [141]:
# extract input and target
X_train = train.drop(['delta_from_nextprice','Traded'],axis=1).values
y_train = train[['Traded']].values.ravel()
X_test = test.drop(['delta_from_nextprice','Traded'],axis=1).values
y_test = test[['Traded']].values.ravel()

In [142]:
# train model
model_1 = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

t0 = dt.datetime.utcnow()
model_1.fit(X_train, y_train, verbose=False)
t1 = dt.datetime.utcnow()

#### Second Model - Predict + and - PnL

In [143]:
# extract input and target
y_train_p = train[['delta_from_nextprice']].values.ravel()
y_test_p = test[['delta_from_nextprice']].values.ravel()

In [91]:
# train model
model_2 = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

t0 = dt.datetime.utcnow()
model_2.fit(X_train, y_train_p, verbose=False)
t1 = dt.datetime.utcnow()

ValueError: Unknown label type: continuous

#### Get Probability for each test set

In [92]:
columns = test.columns
columns

Index(['Side', 'Notional', 'MidPrice', 'QuotedPrice', 'Competitors', 'Traded',
       'delta_from_mid', 'Counterparty_Ctpy_0', 'Counterparty_Ctpy_1',
       'Counterparty_Ctpy_2', 'Counterparty_Ctpy_3', 'Bond_Bond_0',
       'Bond_Bond_1', 'Bond_Bond_2', 'PnL'],
      dtype='object')

In [93]:
suggested_delta = []
for idx,row in test.iterrows():
    sample,delta = max_pnl_price(np.arange(0,1.5,0.01),row,columns,model_1,model_2,1.5)
    suggested_delta.append(delta*row['MidPrice'])
    print(sample)
    print(delta)
    break

     delta_from_mid  Side  Notional  MidPrice  QuotedPrice  Competitors  \
0              1.49   1.0  320000.0     108.2       107.89          2.0   
1              1.48   1.0  320000.0     108.2       107.89          2.0   
2              1.47   1.0  320000.0     108.2       107.89          2.0   
3              1.46   1.0  320000.0     108.2       107.89          2.0   
4              1.45   1.0  320000.0     108.2       107.89          2.0   
..              ...   ...       ...       ...          ...          ...   
295           -1.46   1.0  320000.0     108.2       107.89          2.0   
296           -1.47   1.0  320000.0     108.2       107.89          2.0   
297           -1.48   1.0  320000.0     108.2       107.89          2.0   
298           -1.49   1.0  320000.0     108.2       107.89          2.0   
299           -1.50   1.0  320000.0     108.2       107.89          2.0   

     Traded  Counterparty_Ctpy_0  Counterparty_Ctpy_1  Counterparty_Ctpy_2  \
0       0.0          

In [71]:
pnl(suggested_delta,oos)

pos_pnl:483
better_price:517
traded_pos_pnl:0
